In [2]:
import pandas
import os
from nltk import pos_tag, word_tokenize
from nltk.corpus import sentiwordnet as swn

In [2]:
def get_word_score(word):
    score = 0
    score_positive = 0
    score_negative = 0
    senti = list(swn.senti_synsets(word))
    if(len(senti) > 0):
        for s in senti:
            score_positive += s.pos_score()
            score_negative += s.neg_score()
        score_positive /= len(senti)
        score_negative /= len(senti)
        score = score_positive - score_negative
    return score

In [3]:
def word_is_registered(array, word):
    for a in array:
        if(a == word):
            return True
        
    return False

In [4]:
def scoring_data(data):
    hasil = {}
    hasil['score'] = {}
    hasil['words'] = []
    
    for data in data[0]:
        text = word_tokenize(data)
        postag = pos_tag(text)
        
        #ambil NN, RB, JJ
        noun = []
        word_score = 0
        for tagged in postag:
            if(tagged[1] == 'NN'):
                noun.append(tagged[0])
            else:
                if((tagged[1] == 'RB') or (tagged[1] == 'JJ') or (tagged[1] == 'VBD') or (tagged[1] == 'VBN') or (tagged[1] == 'VB')):
                    word_score += get_word_score(tagged[0])
        for word in noun:
            if(word_is_registered(hasil['words'], word) == False):
                hasil['words'].append(word)
            try:
                hasil['score'][word] += word_score
            except:
                hasil['score'][word] = word_score
    
    return hasil

In [5]:
def sorting_data(scores, labels, word_score):
    hasil = {}
    hasil['words'] = []
    hasil['scores'] = []
    hasil['labels'] = []
    
    #positive
    i = 0
    for word in word_score['words']:
        if(labels[i] == 'Positive'):
            hasil['words'].append(word)
            hasil['scores'].append(scores[i])
            hasil['labels'].append(labels[i])
        i += 1
        
    #negative
    i = 0
    for word in word_score['words']:
        if(labels[i] == 'Negative'):
            hasil['words'].append(word)
            hasil['scores'].append(scores[i])
            hasil['labels'].append(labels[i])
        i += 1
        
    return hasil

In [6]:
data = pandas.read_excel(os.getcwd() +'/Dataset-Canon-S100-Reanotasi.xlsx', header=None)

word_score = scoring_data(data)

scores = []
labels = []
for word in word_score['words']:
    scores.append(word_score['score'][word])
    label = '-'
    if(word_score['score'][word] < 0):
        label = 'Negative'
    else:
        label = 'Positive'
    labels.append(label)
    
sorted_data = sorting_data(scores, labels, word_score)
    
#Export to Excel
raw_data = {'Word': sorted_data['words'],
           'Score': sorted_data['scores'],
           'Label': sorted_data['labels']}
df = pandas.DataFrame(raw_data, columns=['Word', 'Score', 'Label'])
df.to_excel(os.getcwd() +'/word_score.xlsx', sheet_name='Sheet1', index=False)

In [11]:
senti = list(swn.senti_synsets('breakdown'))
print(senti)

#ini cuma ngoutputin buat ngejelasin
print('')
for s in senti:
    print(s)
    print(s.pos_score())
    print(s.neg_score())
    print(s.obj_score())
    print('')

[SentiSynset('dislocation.n.02'), SentiSynset('breakdown.n.02'), SentiSynset('breakdown.n.03'), SentiSynset('breakdown.n.04')]

<dislocation.n.02: PosScore=0.0 NegScore=0.0>
0.0
0.0
1.0

<breakdown.n.02: PosScore=0.125 NegScore=0.5>
0.125
0.5
0.375

<breakdown.n.03: PosScore=0.0 NegScore=0.25>
0.0
0.25
0.75

<breakdown.n.04: PosScore=0.0 NegScore=0.0>
0.0
0.0
1.0

